### Structural Estimation Q2

First, we once again install some required packages and load the data.

In [40]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
%matplotlib inline

In [2]:
ms = pd.read_csv("MacroSeries.txt", header = None, names = ["c","k","w","r"])

Next, we will also define some functions to back out a sequence of z's as well as the pdf of a series of z.

In [111]:
def z3(w, k, alpha):
    ezt = w/((1-alpha)*k**(alpha))
    return np.log(ezt)

def z4(r, k, alpha):
    ezt = r/((alpha)*k**(alpha -1))
    return np.log(ezt)

def pdfz(zvec, mu, sigma, rho):
    probvec = np.zeros_like(zvec)
    probvec[0] = sts.norm.pdf(zvec[0],loc =  mu, scale = sigma)
    for i in range(1,len(zvec)):
        probvec[i] = sts.norm.pdf(zvec[i], loc = rho*(zvec[i-1]) + (1-rho)*mu, scale =  np.sqrt(sigma**2))
    return probvec

def log_like(zvec, mu, sigma, rho):
    pdf_vals = pdfz(zvec, mu, sigma, rho)
    pdf_vals[np.isclose(pdf_vals, np.zeros_like(pdf_vals), atol = 1e-25)] = 1e-10
    log_like = np.log(pdf_vals)
    log_like_sum = np.sum(log_like)
    return log_like_sum


#### (a) 
We will estimate the parameters using MLE first with equations 3 and 4.

In [103]:
def crit1(params, *args):
    alpha, mu, sigma, rho = params
    w, k = args
    zvec = z3(w, k, alpha)
    loglikeval = log_like(zvec, mu, sigma, rho)
    negloglikeval = -loglikeval
    return negloglikeval

In [104]:
crit1((0.4,10,2,0.9), ms['w'], ms['k'])

389.17908571496628

In [114]:
# Intializing Parameters and Minimizing.
args = (ms['w'],ms['k'])
params = 0.3, 9.5, 1, 0.8


results = opt.minimize(crit1, params, args, method = 'L-BFGS-B',
                       bounds=((1e-10, 1), (1e-10, None),(1e-10, None), (-1,1)))
mle3_alpha, mle3_mu, mle3_sigma, mle3_rho = results.x
vc3 = results.hess_inv.todense()
print("MLE for alpha = ", mle3_alpha, "\nMLE for mu = ", mle3_mu, "\nMLE for sigma = ", mle3_sigma, 
     "\nMLE for rho = ", mle3_rho, "\nLikelihood Function = ", results.fun)
print(vc3)

MLE for alpha =  0.457508701955 
MLE for mu =  9.52282593872 
MLE for sigma =  0.0919960448445 
MLE for rho =  0.720493592161 
Likelihood Function =  -96.7069080634
[[  2.12080734e+01  -2.96250555e+02  -3.38465560e-01  -6.05267514e+00]
 [ -2.96250555e+02   4.13826836e+03   4.73522877e+00   8.44791788e+01]
 [ -3.38465560e-01   4.73522877e+00   9.46359382e-03   5.81177494e-02]
 [ -6.05267514e+00   8.44791788e+01   5.81177494e-02   2.10355853e+00]]


#### (b)
We will now estimate the same parameters using equations 4 and 5.

In [115]:
def crit2(params, *args):
    alpha, mu, sigma, rho = params
    r, k = args
    zvec = z4(r, k, alpha)
    loglikeval = log_like(zvec, mu, sigma, rho)
    negloglikeval = -loglikeval
    return negloglikeval

# Intializing Parameters and Minimizing.
args = (ms['r'],ms['k'])
params = 0.3, 9.5, 1, 0.8


results2 = opt.minimize(crit2, params, args,
                       bounds=((1e-10, 1), (1e-10, None),(1e-10, None), (-1,1)))
mle4_alpha, mle4_mu, mle4_sigma, mle4_rho = results.x
vc3 = results.hess_inv.todense()
print("MLE for alpha = ", mle4_alpha, "\nMLE for mu = ", mle4_mu, "\nMLE for sigma = ", mle4_sigma, 
     "\nMLE for rho = ", mle4_rho)
print(vc3)

MLE for alpha =  0.457508701955 
MLE for mu =  9.52282593872 
MLE for sigma =  0.0919960448445 
MLE for rho =  0.720493592161
[[  2.12080734e+01  -2.96250555e+02  -3.38465560e-01  -6.05267514e+00]
 [ -2.96250555e+02   4.13826836e+03   4.73522877e+00   8.44791788e+01]
 [ -3.38465560e-01   4.73522877e+00   9.46359382e-03   5.81177494e-02]
 [ -6.05267514e+00   8.44791788e+01   5.81177494e-02   2.10355853e+00]]


As expected, we get the exact same set of results through the estimation of both equations.
#### (c)
As suggested, let us first solve for $z^* = z_t$ such that $r_t = 1$ by computing:
$$ z^* = \log(\frac{1}{\hat{\alpha} k_t^{\hat{\alpha} - 1}}) $$
We can then find the probability by taking:
$$ P(z_t > z^*) =1 -  F(z^*) $$
where $F(z^*)$ is the CDF of a normal distribution with mean $10\hat{\rho} + (1 - \hat{\rho})\hat{\mu}$ and variance $\hat{\sigma}^2$.

In [116]:
zstar = z4(1, 7500000, mle3_alpha)

prob = 1 - sts.norm.cdf(zstar, 10*mle3_rho + (1 - mle3_rho)*mle3_mu, np.sqrt(mle3_sigma**2))
print("Probability that interest rate is greater than 1 = ", prob)

Probability that interest rate is greater than 1 =  0.999999966734


From our parameters, it appears that the probability that the interest rate today is greater than 1 given a productivity shock of 10 in the previous period and the capital stock currently is 7,500,000 is almost 1.